Практическое занятие 3 Обучение двухслойной ИНС реализации функции исключающее ИЛИ (XOR) с помощью алгоритма обратного распространения

В этом примере кода реализован алгоритм обратного распространения для двухуровневой сети и показано, как использовать его для обучения сети функции исключающее ИЛИ (XOR).

Код инициализации в приведенном ниже фрагменте кода аналогичен тому, что был рассмотрен в примере с обучением персептрона (практическое задание 1). Следует отметить, что мы начали использовать массивы NumPy, чтобы использовать некоторые функции NumPy. То же самое верно и для нашего генератора случайных чисел - мы вызываем np.random.seed вместо просто random.seed. Ответьте, зачем?

Здесь, в обучающих примерах мы изменили истинное значение на значение от 0,0 до 1,0, поскольку для выходного нейрона мы используем теперь логистическую сигмоидальную функцию в качестве функции активации, а её выходной диапазон не доходит до -1,0. Ответьте, почему?

In [1]:
import numpy as np

np.random.seed(3) 
LEARNING_RATE = 0.1
index_list = [0, 1, 2, 3]

x_train = [np.array([1.0, -1.0, -1.0]),
           np.array([1.0, -1.0, 1.0]),
           np.array([1.0, 1.0, -1.0]),
           np.array([1.0, 1.0, 1.0])]
y_train = [0.0, 1.0, 1.0, 0.0] 


Разберите приведённый код, объясните каждую строчку.

В следующем фрагменте кода мы объявляем переменные для хранения состояния трёх нейронов. Реальная реализация обычно параметризуется, чтобы иметь возможность выбирать количество входных данных, слоев и количество нейронов в каждом слое, но в этом примере все эти параметры жестко закодированы, чтобы код был удобен для прочтения.

In [2]:
def neuron_w(input_count):
    weights = np.zeros(input_count+1)
    for i in range(1, (input_count+1)):
        weights[i] = np.random.uniform(-1.0, 1.0)
    return weights

n_w = [neuron_w(2), neuron_w(2), neuron_w(2)]
n_y = [0, 0, 0]
n_error = [0, 0, 0]


Разберите приведённый код, объясните каждую строчку.

Следующий фрагмент кода начинается с функции печати всех девяти весов сети (каждый оператор print печатает трехэлементный вектор весов). Функция forward_pass сначала вычисляет выходные данные нейронов 0 и 1 с одинаковыми входными данными (входными данными из обучающего примера), а затем помещает их выходные данные в массив вместе со значением смещения 1,0 для использования в качестве входных данных для нейрона 2. То есть эта функция определяет топологию сети. Мы используем tanh для нейронов первого слоя и логистическую сигмовидную функцию для выходного нейрона.

Функция reverse_pass начинается с вычисления производной функции ошибок, а затем вычисляет производную функции активации для выходного нейрона. Погрешность выходного нейрона вычисляется путем их умножения. Затем мы продолжаем распространять ошибку на каждый из двух нейронов скрытого слоя. Это делается путем вычисления производных их функций активации и умножения этих производных на член ошибки выходного нейрона и на вес выходного нейрона.

Наконец, функция Adjust_Weights корректирует веса для каждого из трех нейронов. Поправочный коэффициент вычисляется путем умножения входных данных на скорость обучения и погрешность для рассматриваемого нейрона.

In [3]:
def show_learning():
    print('Current weights:')
    for i, w in enumerate(n_w):
        print('neuron ', i, ': w0 =', '%5.2f' % w[0],
              ', w1 =', '%5.2f' % w[1], ', w2 =',
              '%5.2f' % w[2])
    print('----------------')

def forward_pass(x):
    global n_y
    n_y[0] = np.tanh(np.dot(n_w[0], x)) 
    n_y[1] = np.tanh(np.dot(n_w[1], x)) 
    n2_inputs = np.array([1.0, n_y[0], n_y[1]]) 
    z2 = np.dot(n_w[2], n2_inputs)
    n_y[2] = 1.0 / (1.0 + np.exp(-z2))

def backward_pass(y_truth):
    global n_error
    error_prime = -(y_truth - n_y[2]) 
    derivative = n_y[2] * (1.0 - n_y[2]) 
    n_error[2] = error_prime * derivative
    derivative = 1.0 - n_y[0]**2
    n_error[0] = n_w[2][1] * n_error[2] * derivative
    derivative = 1.0 - n_y[1]**2 
    n_error[1] = n_w[2][2] * n_error[2] * derivative

def adjust_weights(x):
    global n_w
    n_w[0] -= (x * LEARNING_RATE * n_error[0])
    n_w[1] -= (x * LEARNING_RATE * n_error[1])
    n2_inputs = np.array([1.0, n_y[0], n_y[1]])
    n_w[2] -= (n2_inputs * LEARNING_RATE * n_error[2])


Разберите приведённый выше код, объясните каждую строчку. Составьте алгоритм по этому коду.

Осталось реализовать последнюю часть кода — это цикл обучения, показанный во фрагменте кода ниже, который напомниает цикл обучения для примера с персептроном из первого практического задания.
Мы выбираем обучающие примеры в случайном порядке, вызываем функции forward_pass, back_pass и Adjust_weights, а затем печатаем значения весов с помощью функции show_learning.

Разберите приведённый ниже код, объясните каждую строчку. Составьте алгоритм по этому коду.

In [5]:
# Network training loop.
all_correct = False
while not all_correct: # Train until converged
    all_correct = True
    np.random.shuffle(index_list) # Randomize order
    for i in index_list: # Train on all examples
        forward_pass(x_train[i])
        backward_pass(y_train[i])
        adjust_weights(x_train[i])
        show_learning() # Show updated weights
    for i in range(len(x_train)): # Check if converged
        forward_pass(x_train[i])
        print('x1 =', '%4.1f' % x_train[i][1], ', x2 =',
              '%4.1f' % x_train[i][2], ', y =',
              '%.4f' % n_y[2])
        if(((y_train[i] < 0.5) and (n_y[2] >= 0.5))
                or ((y_train[i] >= 0.5) and (n_y[2] < 0.5))):
            all_correct = False


Current weights:
neuron  0 : w0 =  0.69 , w1 =  0.78 , w2 =  0.76
neuron  1 : w0 =  0.40 , w1 = -0.58 , w2 = -0.56
neuron  2 : w0 = -0.44 , w1 =  1.02 , w2 =  0.88
----------------
Current weights:
neuron  0 : w0 =  0.70 , w1 =  0.77 , w2 =  0.77
neuron  1 : w0 =  0.41 , w1 = -0.59 , w2 = -0.55
neuron  2 : w0 = -0.43 , w1 =  1.02 , w2 =  0.89
----------------
Current weights:
neuron  0 : w0 =  0.70 , w1 =  0.77 , w2 =  0.77
neuron  1 : w0 =  0.40 , w1 = -0.59 , w2 = -0.56
neuron  2 : w0 = -0.45 , w1 =  1.01 , w2 =  0.89
----------------
Current weights:
neuron  0 : w0 =  0.71 , w1 =  0.77 , w2 =  0.76
neuron  1 : w0 =  0.41 , w1 = -0.59 , w2 = -0.57
neuron  2 : w0 = -0.44 , w1 =  1.01 , w2 =  0.90
----------------
x1 = -1.0 , x2 = -1.0 , y = 0.4244
x1 = -1.0 , x2 =  1.0 , y = 0.6306
x1 =  1.0 , x2 = -1.0 , y = 0.6277
x1 =  1.0 , x2 =  1.0 , y = 0.4976
